<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Send leads and scoring from content interactions to HubSpot
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Send_weekly_post_engagement_metrics_by_email.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #posts #interactions #metrics #analytics #automation #naas #hubspot #sales

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

This templates will get your LinkedIn leads from content likes, comments, use algorithm to score them, send them automatically in notes in HubSpot.

## Input

### Import libraries

In [ ]:
import naas
import pandas as pd
from datetime import datetime
from naas_drivers import linkedin, hubspot
import urllib.parse
import requests
import json
from pprint import pprint

### Setup LinkedIn
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# LinkedIn cookies
LI_AT = "ENTER_YOUR_COOKIE_HERE" # EXAMPLE : "AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2"
JSESSIONID = "ENTER_YOUR_JSESSIONID_HERE" # EXAMPLE : "ajax:8379907400220387585"

# LinkedIn profile url
PROFILE_URL = "ENTER_YOUR_LINKEDIN_PROFILE_HERE" # EXAMPLE "https://www.linkedin.com/in/myprofile/"

# The first execution all posts will be retrieved.
# Then, you can use the parameter below to setup the number of posts you want to retrieved from LinkedIn API everytime this notebook is run.
NO_POSTS_RETRIEVED = 10

### Setup Outputs
Create CSV to store your posts interactions 

In [ ]:
# Custom Path of your CSV with profile URL
profile_id = PROFILE_URL.split("https://www.linkedin.com/in/")[-1].split("/")[0]
profile_id = urllib.parse.unquote(profile_id)

# Inputs
csv_posts = f"LINKEDIN_POSTS_{profile_id}.csv"
csv_likes = f"LINKEDIN_POSTS_LIKES_{profile_id}.csv"
csv_comments = f"LINKEDIN_POSTS_COMMENTS_{profile_id}.csv"

# Outputs
csv_interactions = f"LINKEDIN_INTERACTIONS_{profile_id}.csv"
csv_scoring = f"LINKEDIN_SCORING_{profile_id}.csv"
csv_contacts = f"HUBSPOT_CONTACTS_{profile_id}.csv"

### Setup your HubSpot
- Access your [HubSpot API key](https://knowledge.hubspot.com/integrations/how-do-i-get-my-hubspot-api-key)
- How to find HubSpot [internal field names](https://support.sakari.io/hc/en-us/articles/360038865772-How-to-Find-HubSpot-Internal-Field-Names) ?
- [Create your scoring field in HubSpot](https://knowledge.hubspot.com/crm-setup/manage-your-properties)

In [ ]:
# Enter Token API
HS_API_KEY = "ENTER_YOUR_COOKIE_HERE" # EXAMPLE : "7865b95b-7731-7843-2537-34284HSKHEZ"

# Linkedin field or URL (internal name)
HS_LINKEDIN_FIELD = "ENTER_YOUR_LINKEDIN_FIELD_HERE" # EXAMPLE : "linkedinbio"
HS_SCORING_FIELD = "ENTER_YOUR_SCORING_FIELD_HERE" # EXAMPLE : "scoring"

### Setup Naas scheduler

In [ ]:
# the default settings below will make the notebook run everyday at 8:00
# for information on changing this setting, please check https://crontab.guru/ for information on the required CRON syntax 
naas.scheduler.add(cron="0 8 * * *")

# this notebook will run each week until de-scheduled
# to de-schedule this notebook, simply run the following command: 
# naas.scheduler.delete()

## Model

### Get your posts from CSV
All your posts interactions will be stored in CSV.

In [ ]:
def get_csv(file_path):
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError as e:
        # Empty dataframe returned
        return pd.DataFrame()
    print(f"Dataframe: {len(df)} rows")
    return df

In [ ]:
df_posts = get_csv(csv_posts)
df_posts.head(1)

### Get who likes your posts

In [ ]:
df_likes = get_csv(csv_likes)
df_likes.head(1)

### Get who comments your posts

In [ ]:
df_comments = get_csv(csv_comments)
df_comments.head(1)

### Get contacts from HubSpot
This step returns all contacts in HubSpot with contact id and linkedin value

In [ ]:
df_contacts = hubspot.connect(HS_API_KEY).contacts.get_all(columns=["hs_object_id", HS_LINKEDIN_FIELD, HS_SCORING_FIELD])
print(f"{len(df_contacts)} contacts fetched in HubSpot.")
df_contacts.head(1)

### Update posts
It will get the last X posts from LinkedIn API (X = number of set in variable "NO_POSTS_RETRIEVED") and update it in your CSV.<br>
PS: On the first execution all posts will be retrieved.

In [ ]:
def update_posts(df_posts,
                 csv_output,
                 key="POST_URL",
                 no_posts=10,
                 min_updated_time=60):
    # Init output
    df = pd.DataFrame()
    df_new = pd.DataFrame()
    
    # Init df posts is empty then return entire database
    if len(df_posts) > 0:
        if "DATE_EXTRACT" in df_posts.columns:
            last_update_date = df_posts["DATE_EXTRACT"].max()
            time_last_update = datetime.now() - datetime.strptime(last_update_date, "%Y-%m-%d %H:%M:%S")
            minute_last_update = time_last_update.total_seconds() / 60
            if minute_last_update > min_updated_time:
                # If df posts not empty get the last X posts (new and already existing)
                df_new = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(PROFILE_URL,
                                                                                    limit=no_posts,
                                                                                    sleep=False)
                df_new["DATE_EXTRACT"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            else:
                print(f"🛑 Nothing to update. Last update done {int(minute_last_update)} minutes ago.")
                print(f"✅ {len(df_posts)} posts fetched.")
                return df_posts
    else:
        df_new = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(PROFILE_URL,
                                                                            limit=-1)
        df_new["DATE_EXTRACT"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Concat, save database in CSV and dependency in production
    df = pd.concat([df_new, df_posts]).drop_duplicates(key, keep="first")
    df.to_csv(csv_output, index=False)
    naas.dependency.add(csv_output)

    # Return all posts
    print(f"✅ {len(df)} posts fetched.")
    return df.reset_index(drop=True)

df_posts = update_posts(df_posts,
                        csv_posts,
                        no_posts=NO_POSTS_RETRIEVED)
df_posts.head(1)

### Update likes
It will update your last X (X = number of set in variable "NO_POSTS_RETRIEVED") posts like's from LinkedIn API.<br>
PS: On the first execution all posts like's will be retrieved.

In [ ]:
# Get interactions
def get_interactions(interaction, post_url):
    df = pd.DataFrame()
    if interaction == "LIKES":
        df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(post_url)
    elif interaction == "COMMENTS":
        df = linkedin.connect(LI_AT, JSESSIONID).post.get_comments(post_url)
    return df

In [ ]:
def update_interactions(df_posts,
                        df_interaction,
                        interaction,
                        csv_output,
                        no_posts=10):
    # Init
    df_out = df_interaction.copy()
        
    # Get all interactions if dataframe init empty or not complete
    if len(df_interaction) > 0:
        df_posts = df_posts[:no_posts]
    
    # Loop on posts
    for index, row in df_posts.iterrows():
        df_update = pd.DataFrame()
        post_title = row.TITLE
        post_author = row.AUTHOR_NAME
        post_url = row.POST_URL
        post_date = row.PUBLISHED_DATE
        count_interactions = row[interaction]
        print(f"🔄 {index+1} - Update started on: '{post_title}' ({post_url})")
        
        # Get interactions from post URL
        if len(df_interaction) > 0:
            tmp_df = df_interaction[df_interaction.POST_URL == post_url]
            no_interactions = len(tmp_df)
            if count_interactions != no_interactions:
                print(f"--> {count_interactions} post interaction count vs {no_interactions} interactions.")
                df_update = get_interactions(interaction, post_url)
            else:
                print("--->🛑 Nothing to update.")
        else:
            df_update = get_interactions(interaction, post_url)
        
        # Concat dataframe and save dataframe in CSV
        if len(df_update) > 0:
            print(f"---> {len(df_update)} interactions fetched.")
            df_update['TITLE'] = post_title
            df_update['AUTHOR_NAME'] = post_author
            df_update['PUBLISHED_DATE'] = post_date
            df_update["DATE_EXTRACT"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            keys = ["POST_URL", "PROFILE_ID"]
            if interaction == "COMMENTS":
                keys = ["POST_URL", "PROFILE_ID", "CREATED_TIME"]
            df_out = pd.concat([df_update, df_out]).drop_duplicates(keys, keep="first")
            df_out.to_csv(csv_output, index=False)
            
    # Add dependency in production
    print(f"✅ {len(df_out)} '{interaction}' fetched.")
    naas.dependency.add(csv_output)

    # Return all interactions
    return df_out.reset_index(drop=True)

In [ ]:
df_likes = update_interactions(df_posts,
                               df_likes,
                               "LIKES",
                               csv_likes,
                               NO_POSTS_RETRIEVED)
df_likes.head(1)

### Update comments
It will update your last X (X = number of set in variable "NO_POSTS_RETRIEVED") posts comment's from LinkedIn API.<br>
PS: On the first execution all posts comment's will be retrieved.

In [ ]:
df_comments = update_interactions(df_posts,
                                  df_comments,
                                  "COMMENTS",
                                  csv_comments,
                                  NO_POSTS_RETRIEVED)
df_comments.head(1)

### Create scoring database
- Get LIKES and COMMENTS by profile
- Calculate SCORE : LIKES + COMMENTS
- Add last interaction information

In [ ]:
def create_scoring_db(df_likes, df_comments):
    # Init outputs
    df = pd.DataFrame()
    
    # Dataframe likes
    df1 = df_likes.groupby("PROFILE_ID", as_index=False).agg({"POST_URL": "count"})
    df1 = df1.rename(columns={"POST_URL": "LIKES"})

    # Dataframe comments
    df2 = df_comments.groupby("PROFILE_ID", as_index=False).agg({"POST_URL": "count"})
    df2 = df2.rename(columns={"POST_URL": "COMMENTS"})
    
    # Merge dataframe
    df = pd.merge(df1, df2, on="PROFILE_ID", how="outer").fillna(0)
    
    # Calc score
    df["SCORE"] = df["LIKES"] + df["COMMENTS"]
    
    # Group by profile
    to_keep = [
        "PROFILE_ID",
        "PROFILE_URL",
        "PUBLIC_ID",
        "FIRSTNAME",
        "LASTNAME",
        "FULLNAME",
        "OCCUPATION",
        "PROFILE_TYPE",
        "POST_URL",
        "AUTHOR_NAME",
        "TITLE",
        "PUBLISHED_DATE"
    ]
    df_meta1 = df_likes[to_keep]
    df_meta2 = df_comments[to_keep]
    df_meta = pd.concat([df_meta1, df_meta2]).drop_duplicates("PROFILE_ID", keep="first")
    to_rename = {
        "POST_URL": "LAST_INTERACTION_POST_URL",
        "PUBLISHED_DATE": "LAST_INTERACTION_DATE",
        "TITLE": "LAST_INTERACTION_POST_TITLE",
    }
    df_meta = df_meta.rename(columns=to_rename)
    
    # Merge data to get meta
    df = pd.merge(df_meta, df, on="PROFILE_ID")
    df = df.sort_values(by="SCORE", ascending=False)
    print(f"✅ {len(df)} profiles fetched.")
    return df.reset_index(drop=True)

df_scoring = create_scoring_db(df_likes, df_comments)
df_scoring.head(3)

### Add HubSpot contact ID to LinkedIn
If LinkedIn PROFILE_ID or PUBLIC_ID match with HS_LINKEDIN_FIELD in HubSpot, contact ID will be returned for contact.<br>
Otherwise, a new contact will be created in HubSpot.

In [ ]:
def add_hs_contact_id(df_linkedin, df_hubspot):
    # Cleaning
    df_hubspot[HS_LINKEDIN_FIELD] = df_hubspot[HS_LINKEDIN_FIELD].fillna("TBU")
    df_hubspot[HS_SCORING_FIELD] = df_hubspot[HS_SCORING_FIELD].fillna(0)
    df_linkedin = df_linkedin.reset_index(drop=True)
    
    # Loop in interaction dataframe
    for index, row in df_linkedin.iterrows():
        contact_id = ""
        score = 0
        fullname = row.FULLNAME
        profile_id = row.PROFILE_ID
        public_id = row.PUBLIC_ID
        profile_url = row.PROFILE_URL
        print(f"{index+1} - ➡️ Checking : {fullname} ({public_id})")
        
        # Check profile ID, public ID and encoding public ID
        profile_ids = [public_id, profile_id]
        encode_public_id = urllib.parse.quote(public_id)
        if encode_public_id != public_id:
            profile_ids = profile_ids + [encode_public_id]
        df_profile = pd.DataFrame()
        for profile in profile_ids:
            tmp_df = df_hubspot[df_hubspot[HS_LINKEDIN_FIELD].str.match(f".+{profile}+")].reset_index(drop=True)
            df_profile = pd.concat([df_profile, tmp_df]).reset_index(drop=True)
            
        # Get contact ID(s)
        if len(df_profile) > 0:
            contact_ids = df_profile["hs_object_id"].unique().tolist()
            score = df_profile.loc[0, HS_SCORING_FIELD]
            
            # Update LinkedIn field with url with PROFILE_ID => PUBLIC ID can be change by user but PUBLIC_ID is unique
            for i, r in df_profile.iterrows():
                hs_id = r["hs_object_id"]
                lk_field = r[HS_LINKEDIN_FIELD]
                if profile_id not in lk_field:
                    hubspot.connect(HS_API_KEY).contacts.patch(hs_id, {"properties": {HS_LINKEDIN_FIELD: profile_url}})
            
            # List to string
            contact_id = ",".join(contact_ids)
        else:
            contact = {
                "properties": {
                    "firstname": row.FIRSTNAME,
                    "lastname": row.LASTNAME,
                    "jobtitle": row.OCCUPATION,
                    HS_LINKEDIN_FIELD: row.PROFILE_URL,
                    HS_SCORING_FIELD: score,
                }
            }
            contact_id = hubspot.connect(HS_API_KEY).contacts.send(contact)
            
        # Add contact ID(s) to LinkedIn profile
        df_linkedin.loc[index, "HS_CONTACT_ID"] = contact_id
        df_linkedin.loc[index, "HS_SCORE"] = score

    # Save dataframe to csv
    df_linkedin["DATE_EXTRACT"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    df_linkedin.to_csv(csv_interactions, index=False)
    return df_linkedin

db_scoring = add_hs_contact_id(df_scoring, df_contacts)
db_scoring

### Create interactions database
- Concat LIKES and COMMENTS in single database
- Create note for HubSpot

In [ ]:
def create_interactions_db(df_likes, df_comments):
    # Init outputs
    df = pd.DataFrame()
    
    # Dataframe likes
    df_likes["REACTION"] = "LIKES"
    df_likes["NOTE"] = "Liked on '" + df_likes["TITLE"]  + "'"
    df_likes["NOTE_HTML"] = "Liked on <a href='" + df_likes["POST_URL"] + "'>'" + df_likes["TITLE"]  + "'</a>"

    # Dataframe comments
    df_comments["REACTION"] = "COMMENTS"
    df_comments["NOTE"] = "Commented '" + df_comments["TEXT"] + "' on '" + df_comments["TITLE"] + "'"
    df_comments["NOTE_HTML"] = "Commented '" + df_comments["TEXT"] + "' on <a href='" + df_comments["POST_URL"] + "'>'" + df_comments["TITLE"]  + "'</a>"
    
    # Concat
    df = pd.concat([df_likes, df_comments]).fillna("Not defined").sort_values(by="PUBLISHED_DATE", ascending=False)
    
    # Cleaning
    to_keep = [
        "PROFILE_ID",
        "PROFILE_URL",
        "PUBLIC_ID",
        "FIRSTNAME",
        "LASTNAME",
        "FULLNAME",
        "REACTION",
        "TEXT",
        "TITLE",
        "PUBLISHED_DATE",
        "AUTHOR_NAME",
        "POST_URL",
        "NOTE",
        "NOTE_HTML"
    ]
    df = df[to_keep]
    
    print(f"✅ {len(df)} interactions fetched.")
    return df.reset_index(drop=True)

df_interactions = create_interactions_db(df_likes, df_comments)
df_interactions.head(3)

## Output

### Create note with association

In [ ]:
def create_note(content, contactids=None, dealids=None, companyids=None, ownerid=None, date_time=None):
    # Setup timestamp
    if date_time is not None:
        timestamp = str(int(datetime.strptime(date_time[:19], "%Y-%m-%d %H:%M:%S").timestamp())) + "000"
    else:
        timestamp = str(int(datetime.now().timestamp())) + "000"
        
    # Setup payload
    payload = json.dumps(
        {
            "engagement": {
                "active": "true",
                "ownerId": ownerid,
                "type": "NOTE",
                "timestamp": timestamp,
            },
            "associations": {
                "contactIds": [contactids],
                "dealIds": [dealids],
                "companyIds": [companyids],
            },
            "attachments": [{}],
            "metadata": {"body": content},
        }
    )
    headers = {
            "accept": "application/json",
            "content-type": "application/json",
        }
    params = {"limit": "100",
              "archived": "false",
              "hapikey": HS_API_KEY}
    url = "https://api.hubapi.com/engagements/v1/engagements"
    # Requests data
    res = requests.post(url,
                        data=payload,
                        headers=headers,
                        params=params,
                        allow_redirects=False)
    res.raise_for_status()
    res_json = res.json()
    # Note ID
    note_id = res_json.get("engagement").get("id")
    # Message success
    print(f"✅ New note created '{note_id} 'in HubSpot: {content}")
    return note_id0000

### Get notes from contacts

In [ ]:
def get_notes_from_contact(contact_id):
    url = f'https://api.hubapi.com/engagements/v1/engagements/associated/contact/{contact_id}/paged'
    querystring = {
        "archived": "false",
        "hapikey": HS_API_KEY,
        "limit": 100,
    }
    headers = {'accept': 'application/json'}

    # Get all notes
    df_notes = pd.DataFrame()
    engagements = []
    has_more = True
    offset = None
    while has_more:
        if offset is not None:
            querystring["offset"] = offset
            
        # Requests data
        res = requests.get(url,
                           headers=headers,
                           params=querystring)
        res.raise_for_status()
        res_json = res.json()
        results = res_json.get("results")
        if len(results) > 0:
            for result in results:
                uid = result.get("engagement").get("id")
                content = result.get("engagement").get("bodyPreview")
                content_type = result.get("engagement").get("type")
                timestamp = result.get("engagement").get("timestamp")
                engagement = {
                    "UID": uid,
                    "TIMESTAMP": timestamp,
                    "CONTENT": content,
                    "TYPE": content_type
                }
                engagements.append(engagement)
        has_more = res_json.get("hasMore")
        offset = res_json.get("offset")
        
    df_engagements = pd.DataFrame(engagements)
    if len(df_engagements) > 0:
        df_notes = df_engagements[df_engagements["TYPE"] == "NOTE"].sort_values("TIMESTAMP").reset_index(drop=True)
    print(f"✅ {len(df_notes)} notes fetched from contact {contact_id}")
    return df_notes

### Update contact score and send interactions to notes
- Update score to profile
- Add new notes if does not already exists

In [ ]:
def update_hs_contact(df_score, df_interactions, public_id):
    # Init output
    df_out = pd.DataFrame()
    
    # Filter to get updated score and exclude yourself
    df = df_score[(df_score["SCORE"].astype(float) != df_score["HS_SCORE"].astype(float)) & 
                  (df_score["PUBLIC_ID"] != public_id)].reset_index(drop=True)
    if len(df) == 0:
        print("🛑 Nothing to update.")
        return df

    # Loop on dataframe
    for index, row in df.iterrows():
        # Init variables
        contact_ids = row.HS_CONTACT_ID
        fullname = row.FULLNAME
        hs_score = row.HS_SCORE
        score = row.SCORE
        profile_id = row.PROFILE_ID
        print(f"{index+1} - ➡️ Checking : {fullname} ({contact_ids})")
        
        # Split and iterate on contact IDs
        hs_ids = contact_ids.split(",")
        for hs_id in hs_ids:
            print(f"--> starting with HubSpot ID: {hs_id}")
            # -> Update score
            if hs_id != "":
                if float(hs_score) != float(score):
                    hubspot.connect(HS_API_KEY).contacts.patch(hs_id, {"properties": {HS_SCORING_FIELD: int(score)}})
                
                #-> Send interactions to notes
                # Create interactions dataframe
                tmp_df = df_interactions[df_interactions["PROFILE_ID"] == profile_id].reset_index(drop=True)
                
                # Get notes from contact
                df_notes = get_notes_from_contact(hs_id)

                # Add all interactions if does not already exist
                hs_notes = []
                for i, r in tmp_df.iterrows():
                    note = r.NOTE
                    note_html = r.NOTE_HTML
                    date = r.PUBLISHED_DATE
                    if len(df_notes) > 0:
                        hs_notes = [x.replace(" ", "") for x in df_notes["CONTENT"].unique().tolist() if x is not None] 
                        
                    if note.replace(" ", "") not in hs_notes:
                        create_note(note_html, contactids=hs_id, date_time=date)
                df_out = pd.concat([df_out, tmp_df])
    return df_out

df_leads = update_hs_contact(db_scoring, df_interactions, profile_id)
df_leads

### Save scoring dataframe to csv

In [ ]:
df_scoring.to_csv(csv_scoring, index=False)

### Save interactions dataframe to csv

In [ ]:
df_interactions.to_csv(csv_interactions, index=False)